### 0. Setting

BERT.ipynb와 동일

In [1]:
import os
import urllib.request
import zipfile
import tarfile
import glob
import io

In [2]:
# data
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

target_dir_path="./data/"
if not os.path.exists(target_dir_path):
    os.mkdir(target_dir_path)
    
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
save_path = "./data/aclImdb_v1.tar.gz"
urllib.request.urlretrieve(url, save_path)

tar = tarfile.open('./data/aclImdb_v1.tar.gz')
tar.extractall('./data/') 
tar.close()  


# vocab
vocab_dir = "./vocab/"
if not os.path.exists(vocab_dir):
    os.mkdir(vocab_dir)

save_path="./vocab/bert-base-uncased-vocab.txt"
url = "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt"
urllib.request.urlretrieve(url, save_path)


# weights
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

save_path = "./weights/bert-base-uncased.tar.gz"
url = "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz"
urllib.request.urlretrieve(url, save_path)

archive_file = "./weights/bert-base-uncased.tar.gz"  
tar = tarfile.open(archive_file, 'r:gz')
tar.extractall('./weights/')  
tar.close()

In [3]:
target_dir_path="./data/aclImdb/"

if os.path.exists(target_dir_path):
    # Train
    f = open('./data/IMDb_train.tsv','w')

    path = './data/aclImdb/train/pos/'
    for fname in glob.glob(os.path.join(path,'*.txt')):
        with io.open(fname, 'r', encoding="utf-8") as ff:
            text = ff.readline()
            text = text.replace('\t', " ")
            text = text+'\t'+'1'+'\t'+'\n'
            f.write(text)

    path = './data/aclImdb/train/neg/'
    for fname in glob.glob(os.path.join(path,'*.txt')):
        with io.open(fname, 'r', encoding="utf-8") as ff:
            text = ff.readline()
            text = text.replace('\t', " ")
            text = text+'\t'+'0'+'\t'+'\n'
            f.write(text)

    f.close()
    
    # Test
    f = open('./data/IMDb_test.tsv','w')

    path = './data/aclImdb/test/pos/'
    for fname in glob.glob(os.path.join(path,'*.txt')):
        with io.open(fname, 'r', encoding="utf-8") as ff:
            text = ff.readline()
            text = text.replace('\t', " ")
            text = text+'\t'+'1'+'\t'+'\n'
            f.write(text)

    path = './data/aclImdb/test/neg/'

    for fname in glob.glob(os.path.join(path,'*.txt')):
        with io.open(fname, 'r', encoding="utf-8") as ff:
            text = ff.readline()
            text = text.replace('\t', " ")        
            text = text+'\t'+'0'+'\t'+'\n'
            f.write(text)

    f.close()

In [4]:
import json

# Config
config_file = "./weights/bert_config.json"

json_file = open(config_file, 'r')
config = json.load(json_file)

config

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [5]:
! pip install attrdict

In [6]:
! git clone https://github.com/gymoon10/utils.git

Cloning into 'utils'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 47 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [7]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext

### 1. DataLoader

In [8]:
import re
import string
from utils.bert import BertTokenizer

def preprocessing_text(text):
    '''IMDb 전처리'''
    text = re.sub('<br />', '', text)

    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# Tokenizer 
tokenizer_bert = BertTokenizer(
    vocab_file="./vocab/bert-base-uncased-vocab.txt", do_lower_case=True)

# Pre-process + Tokenize
def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)  # tokenizer_bert

    return ret

In [9]:
from torchtext.legacy import data

max_length = 256

TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                                   lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="[CLS]", eos_token="[SEP]", pad_token='[PAD]', unk_token='[UNK]')

LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

`sequential` : 데이터의 길이가 가변인지. 문장은 길이가 다양하므로 True. 라벨은 False

`tokenize` : 문장을 읽을 때, 전처리 및 단어 분할 함수를 정의

`use_vocab` : 단어를 vocabulary에 추가할지 여부

`lower` : 알파벳이 존재할 때 소문자로 변환할지 여부

`include_length` : 문장의 단어 수 데이터를 포함할지 여부

`batch_first` : 미니 배치 차원을 선두에 제공할지 여부

`fix_length` : 전체 문장을 지정한 길이가 되도록 padding

`init_token, eos_token, pad_token, unk_token` : 문장 선두, 문장 말미, padding, 미지어에 대해, 어떤 단어를 부여하는지 지정

In [10]:
# "data" 폴더에서 각 tsv 파일을 읽기
from torchtext.legacy.data import TabularDataset

train_val_ds, test_ds = TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# train/val split
train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

In [11]:
from utils.bert import BertTokenizer, load_vocab

# BERT용 Vocab
vocab_bert, ids_to_tokens_bert = load_vocab(
    vocab_file="./vocab/bert-base-uncased-vocab.txt")

TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.stoi = vocab_bert

In [12]:
# DataLoader 작성 (torchtext의 iterater)
from torchtext.legacy.data import Iterator

batch_size = 32  # BERT에서는 16, 32에 가까운 수를 빈번하게 사용

train_dl = Iterator(train_ds, batch_size=batch_size, train=True)

val_dl = Iterator(val_ds, batch_size=batch_size, train=False, sort=False)

test_dl = Iterator(test_ds, batch_size=batch_size, train=False, sort=False)

# dict 객체로 정리
dataloaders_dict = {"train": train_dl, "val": val_dl}

In [13]:
# 동작 확인
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[ 101, 2066, 1996,  ..., 2069, 2138,  102],
        [ 101, 2044, 2035,  ...,    0,    0,    0],
        [ 101, 1045, 1049,  ..., 2204, 1010,  102],
        ...,
        [ 101, 2034, 1997,  ..., 1037, 2843,  102],
        [ 101, 2317, 2158,  ...,    0,    0,    0],
        [ 101, 2023, 2003,  ...,    0,    0,    0]]), tensor([256, 192, 256, 160, 256, 180, 130, 172, 243, 206, 140, 214, 193, 256,
        256, 140, 255, 159, 183, 160, 256, 152, 198, 217, 160, 227, 248, 256,
        102, 256, 170,  62]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 0, 1, 1, 1])


In [14]:
# mini-batch의 첫 번째 문장
text_minibatch_1 = (batch.Text[0][1]).numpy()

# id -> 단어
text = tokenizer_bert.convert_ids_to_tokens(text_minibatch_1)

print(text)

['[CLS]', 'after', 'all', 'these', 'years', ',', 'of', 'peter', 'o', 'tool', 's', 'brilliant', ',', 'costly', 'giving', 'of', 'his', 'soul', ',', 'film', 'after', 'film', ',', 'at', 'last', ',', 'hollywood', 'toss', '##es', 'him', 'an', 'oscar', 'recently', '.', 'country', 'dance', 'showed', 'up', 'one', 'night', 'late', ',', 'and', 'of', 'course', ',', 'blew', 'me', 'out', 'of', 'my', 'complain', '##ant', 'niche', 'in', 'my', 'alleged', 'life', '.', 'how', 'does', 'he', 'do', 'it', 'york', 'again', 'also', 'is', 'brilliant', 'in', 'this', 'kind', 'of', 'play', '.', 'both', 'psychological', 'battleships', 'loaded', 'for', 'bear', '.', '.', '.', '.', 'bravo', 'to', 'author', ',', 'director', ',', 'cast', ',', 'and', 'camera', 'crew', '.', 'no', 'wonder', 'the', 'nazi', 's', 'lost', 'to', 'these', 'irish', ',', 'sc', '##ot', ',', 'english', 'blend', '##s', '.', '.', '.', '.', 'brutal', 'honesty', 'hurts', '.', '.', '.', 'back', 'in', 'the', '70', 's', ',', 'when', 'i', 'personally', 'bel

### 2. Model

![image](https://user-images.githubusercontent.com/44194558/151937826-41048c6b-9bd6-45d6-ae53-f2b6a827fe22.png)


In [15]:
from utils.bert import get_config, BertModel, set_learned_params

# 모델의 config 파일 불러오기
config = get_config(file_path="./weights/bert_config.json")

# BERT 모델을 작성
net_bert = BertModel(config)

# BERT 모델에 학습된 파라미터 설정
net_bert = set_learned_params(
    net_bert, weights_path="./weights/pytorch_model.bin")

bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.gamma→embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.beta→embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight→encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.bias→encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.key.weight→encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.bias→encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.value.weight→encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.bias→encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attention.output.dense.weight→encoder.layer.0.attention.output

In [16]:
class BertForIMDb(nn.Module):
    '''BERT 모델에 IMDb 리뷰에 대한 긍정/부정 이진 분류 태스크를 연결'''
    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()
        
        # BERT 모델
        self.bert = net_bert

        # head에 긍정/부정 이진 분류 예측 추가
        self.cls = nn.Linear(in_features=768, out_features=2)

        # 추가된 layer의 가중치 초기화
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_fig=False):
        '''
        input_ids:  [batch_size, seq_length] 문장의 단어 ID 나열
        token_type_ids:  [batch_size, seq_length] 각 단어가 1번째 문장인지, 2번째 문장인지를 나타내는 id
        attention_mask: masking
        output_all_encoded_layers: BertEncoder를 구성하는 모든 BertLayer의 출력을 반환할 지, 마지막 12번째 층의 출력만 반환할 지
        attention_show_flg: Self-Attention의 가중치를 반환할지의 플래그
        '''
        # BERT forward
        if attention_show_fig == True:
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_fig)
            
        elif attention_show_fig == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_fig)
            
        # 입력 문장의 첫 번째 단어 [CLS] 토큰의 hidden_state를 사용하여 긍정/부정 이진 분류 수행
        CLS_vec = encoded_layers[:, 0, :]  # encoded_layers : [batch, seq_length, hidden_size]
        CLS_vec = CLS_vec.view(-1, 768)  #  [batch, hidden_size]
        out = self.cls(CLS_vec)

        if attention_show_fig == True:
            return out, attention_probs
            
        elif attention_show_fig == False:
            return out

In [17]:
# 모델 구축
net = BertForIMDb(net_bert)
net.train()  # train

print('네트워크 설정 완료')

네트워크 설정 완료


### 3. Fine-tuning

사전 학습된 BERT 모델의 파라미터는 갱신 x

BertEncoder의 마지막 12번 째 BertLayer, 새로 추가한 이진 분류기의 파라미터만 갱신

In [18]:
# 1. 네트워크 전체의 파라미터를 동결
for name, param in net.named_parameters():
    param.requires_grad = False

# 2. 마지막 BertLayer의 파라미터를 동결 해제 (task specific하게 학습될 수 있도록)
for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

# 3. 이진 분류기의 파라미터를 동결 해제
for name, param in net.cls.named_parameters():
    param.requires_grad = True

In [19]:
# Optimizer - 위에서 동결 해제된 파라미터들에만 적용
optimizer = optim.Adam([{'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
                        {'params': net.cls.parameters(), 'lr': 5e-5}], betas=(0.9, 0.999))
# Loss
criterion = nn.CrossEntropyLoss()

### 4. Train & Validation

In [20]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)
    print('-----start-------')

    net.to(device)

    torch.backends.cudnn.benchmark = True

    batch_size = dataloaders_dict["train"].batch_size

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 훈련 모드
            else:
                net.eval()  # 검증 모드   

            epoch_loss = 0.0  # 손실 합
            epoch_corrects = 0  # 정답 수
            iteration = 1

            t_epoch_start = time.time()
            t_iter_start = time.time()

            for batch in (dataloaders_dict[phase]):  # Text, Lable의 dictionary 변수
                inputs = batch.Text[0].to(device)  # 문장
                labels = batch.Label.to(device)  # 라벨

                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    # BertForIMDb
                    outputs = net(inputs, token_type_ids=None, attention_mask=None,
                                  output_all_encoded_layers=False, attention_show_fig=False)
                    # loss
                    loss = criterion(outputs, labels)
                    
                    # prediction
                    _, preds = torch.max(outputs, 1)
                    
                    # Backward (train)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0): 
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec. || 이 반복의 정답률: {}'.format(
                                iteration, loss.item(), duration, acc))
                            t_iter_start = time.time()

                    iteration += 1

                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))
            t_epoch_start = time.time()

    return net

In [21]:
num_epochs = 2
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)

사용 장치:  cuda:0
-----start-------
반복 10 || Loss: 0.6667 || 10iter: 10.8498 sec. || 이 반복의 정답률: 0.65625
반복 20 || Loss: 0.6772 || 10iter: 10.6722 sec. || 이 반복의 정답률: 0.5625
반복 30 || Loss: 0.6406 || 10iter: 10.7190 sec. || 이 반복의 정답률: 0.65625
반복 40 || Loss: 0.6559 || 10iter: 10.7994 sec. || 이 반복의 정답률: 0.59375
반복 50 || Loss: 0.6271 || 10iter: 10.8852 sec. || 이 반복의 정답률: 0.6875
반복 60 || Loss: 0.6022 || 10iter: 10.9527 sec. || 이 반복의 정답률: 0.6875
반복 70 || Loss: 0.4928 || 10iter: 10.9730 sec. || 이 반복의 정답률: 0.90625
반복 80 || Loss: 0.5136 || 10iter: 10.9873 sec. || 이 반복의 정답률: 0.78125
반복 90 || Loss: 0.3224 || 10iter: 10.9926 sec. || 이 반복의 정답률: 0.90625
반복 100 || Loss: 0.3861 || 10iter: 10.9650 sec. || 이 반복의 정답률: 0.8125
반복 110 || Loss: 0.4338 || 10iter: 10.9574 sec. || 이 반복의 정답률: 0.84375
반복 120 || Loss: 0.5856 || 10iter: 10.9836 sec. || 이 반복의 정답률: 0.71875
반복 130 || Loss: 0.3027 || 10iter: 10.9730 sec. || 이 반복의 정답률: 0.875
반복 140 || Loss: 0.2349 || 10iter: 10.9691 sec. || 이 반복의 정답률: 0.9375
반복 150 || Loss: 0

In [22]:
# 학습한 네트워크 파라미터를 저장
save_path = './weights/bert_fine_tuning_IMDb.pth'
torch.save(net_trained.state_dict(), save_path)

In [24]:
# 테스트 데이터의 정답률 계산
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   
net_trained.to(device)  

epoch_corrects = 0
for batch in tqdm(test_dl):  
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 문장
    labels = batch.Label.to(device)  # 라벨

    # Forward
    with torch.set_grad_enabled(False):
        outputs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                              output_all_encoded_layers=False, attention_show_fig=False)

        loss = criterion(outputs, labels)  
        _, preds = torch.max(outputs, 1)  

        epoch_corrects += torch.sum(preds == labels.data)  # 정답수의 합계를 갱신

# 정답률
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('테스트 데이터 {}개에서 정답률: {:.4f}'.format(len(test_dl.dataset), epoch_acc))

100%|██████████| 782/782 [13:32<00:00,  1.04s/it]


테스트 데이터 25000개에서 정답률: 0.9016


### 5. Attention 시각화

In [26]:
batch_size = 64
test_dl = Iterator(test_ds, batch_size=batch_size, train=False, sort=False)

batch = next(iter(test_dl))

inputs = batch.Text[0].to(device)  
labels = batch.Label.to(device)  

outputs, attention_probs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                                       output_all_encoded_layers=False, attention_show_fig=True)

_, preds = torch.max(outputs, 1) 

In [28]:
print(outputs.shape)
print(preds)

torch.Size([64, 2])
tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')


In [29]:
# HTML 작성 함수 구현
def highlight(word, attn):
    "Attention 값이 크면 문자 배경을 진한 빨간색으로 하는 html을 출력하는 함수"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights, TEXT):
    "HTML 데이터를 작성한다"

    # index의 결과를 추출
    sentence = batch.Text[0][index]  # 문장
    label = batch.Label[index]  # 라벨
    pred = preds[index]  # 예측

    # 라벨과 예측 결과를 문자로 대체
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 표시용의 HTML을 작성한다
    html = '정답 라벨: {}<br>추론 라벨: {}<br><br>'.format(label_str, pred_str)

    # Self-Attention의 가중치를 가시화. Multi-Head가 12개이므로, 12종류의 attention이 존재
    for i in range(12):

        # index의 Attention을 추출하고 규격화
        # 0번째 단어 [CLS]의, i번째의 Multi-Head Attention를 꺼내
        # index는 미니 배치의 몇 번째 데이터인지를 나타냄
        attens = normlized_weights[index, i, 0, :]
        attens /= attens.max()

        html += '[BERT의 Attention을 시각화_' + str(i+1) + ']<br>'
        for word, attn in zip(sentence, attens):

            # 단어가 [SEP]인 경우 문장의 끝이므로 break
            if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
                break

            # 함수 highlight로 색을 칠하고, 함수 tokenizer_bert.convert_ids_to_tokens로 ID를 단어로 되돌림
            html += highlight(tokenizer_bert.convert_ids_to_tokens(
                [word.numpy().tolist()])[0], attn)
        html += "<br><br>"

    # 12종류의 Attention의 평균을 구한다. 최대치로 규격화
    all_attens = attens*0  # all_attens이라는 변수를 작성한다
    for i in range(12):
        attens += normlized_weights[index, i, 0, :]
    attens /= attens.max()

    html += '[BERT의 Attention을 시각화_ALL]<br>'
    for word, attn in zip(sentence, attens):

        # 단어가 [SEP]일 경우는 문장의 끝이므로 break
        if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
            break

        # 함수 highlight로 색을 칠하고, 함수 tokenizer_bert.convert_ids_to_tokens로 ID를 단어로 되돌림
        html += highlight(tokenizer_bert.convert_ids_to_tokens(
            [word.numpy().tolist()])[0], attn)
    html += "<br><br>"

    return html

In [30]:
from IPython.display import HTML

index = 3  # 출력할 데이터
html_output = mk_html(index, batch, preds, attention_probs, TEXT)  # HTML 작성
HTML(html_output) 

In [31]:
from IPython.display import HTML

index = 61  # 출력할 데이터
html_output = mk_html(index, batch, preds, attention_probs, TEXT)  # HTML 작성
HTML(html_output) 


![image](https://user-images.githubusercontent.com/44194558/151950444-c07277ce-effb-49d0-8e01-b2fc4acb8928.png)
